In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("udf").getOrCreate()

22/04/04 18:09:13 WARN Utils: Your hostname, devkhk-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.30.1.27 instead (on interface en0)
22/04/04 18:09:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 18:09:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
s

In [4]:
df = spark.createDataFrame(data=transactions, schema=schema)

In [5]:
df.createOrReplaceTempView("transactions")

In [6]:
spark.sql("select * from transactions").show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



In [7]:
# UDF

In [8]:
# 아래 함수를 스파크 sql에서 사용하기 위해서 레지스터 등록을 해야한다.
# 레지스터 등록은 스파크 세션에서 할 수 있다.
from pyspark.sql.types import LongType
def squared(n):
    return n * n

spark.udf.register("squared", squared, LongType())

<function __main__.squared(n)>

In [9]:
spark.sql("SELECT price, squared(price) FROM transactions").printSchema()
# udf 레지스터에 등록된 함수의 리턴값의 데이터 타입이 정의되지 않으면 디폴트값으로 스트링값이 된다.

root
 |-- price: long (nullable = true)
 |-- squared(price): long (nullable = true)



In [10]:
# udf 등록 두번째 방식
# 조금더 파이썬스러운 코드
# 스파크 sql문을 실행시키면 똑같이 동작하는것이 맞지만 현재 undefined function 에러가 있다.
# 낮은 버전의 pyspark에서는 공식 버그 리포팅이 있다.
# 해결법으로는 첫번째 방법으로 직접 register 해주는 방법을 알려준다.
# 그러므로 틀린 방법은 아니지만 위 방법을 사용하는것을 권장.

from pyspark.sql.functions import udf

@udf("long")
def udf_squared(n):
    return n * n

In [11]:
def read_number(n) :
    units = ["", "십", "백", "천", "만"]
    nums = '일이삼사오육칠팔구'
    
    result= []
    i = 0
    while n > 0:
        n , r = divmod(n, 10)
        if r > 0:
            result.append(nums[r-1] + units[i])
        i += 1
    return "".join(reversed(result))
    
print(read_number(23123))
print(read_number(72300))

이만삼천일백이십삼
칠만이천삼백


In [12]:
spark.udf.register("read_number", read_number)

<function __main__.read_number(n)>

In [13]:
spark.sql("select price, read_number(price) from transactions").show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|22000|          이만이천|
|21500|      이만일천오백|
|42000|          사만이천|
|21250|  이만일천이백오십|
|68700|      육만팔천칠백|
|32000|          삼만이천|
+-----+------------------+



In [34]:
def get_weekday(date):
    import calendar
    return calendar.day_name[date.weekday()]

spark.udf.register("get_weekday", get_weekday)

22/04/04 18:36:12 WARN SimpleFunctionRegistry: The function get_weekday replaced a previously registered function.


<function __main__.get_weekday(date)>

In [37]:
# TO_DATE : 스파크 내부 함수
query = """
SELECT
    datetime,
    get_weekday(TO_DATE(datetime)) as day_of_week
FROM
    transactions
"""

spark.sql(query).show()

+-------------------+-----------+
|           datetime|day_of_week|
+-------------------+-----------+
|2021-11-07 13:20:00|     Sunday|
|2021-10-24 11:19:00|     Sunday|
|2021-07-25 11:12:40|     Sunday|
|2021-07-10 08:20:00|   Saturday|
|2021-07-01 05:36:00|   Thursday|
|2020-08-19 19:04:00|  Wednesday|
+-------------------+-----------+

